# Omdena  - Milan Chapter Agrifoods
## AI for Sustainable agri-food systems: use of Satellite Imagery
### Exploratory analysis of fruits_crop production in Italy 2006-2022
#### Author: Maria Fisher 


The main objective of this study is to have gather information about crop production in Italy for the period of 2006-2022. 

Crop dataset used in this study was downloaded from the Italian National Institute of Statistics (Istat).



In [ ]:
import warnings 
warnings.filterwarnings("ignore")

import os
import pandas as pd
pd.options.display.float_format = "{:.2f}".format
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns 
import scipy 
import sklearn
import geopandas as gpd
import pgeocode
import folium
import sys
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot

In [ ]:
fruits = pd.read_csv('./Italy_crop_data/fruits_crop.csv',skipinitialspace=True)
fruits.head()

## Pre-processing dataset 

In [ ]:
# Drop Columns
fruits = fruits.drop(columns =['ITTER107','TIPO_DATO5','AGRI_MADRE', 'TIME', 'Flag Codes','Flags' ])
fruits

In [ ]:
# Rename Columns
fruits = fruits.rename(columns = {'Select time':'Year', 'Type of crop':'Type_crop', 'Data type':'Data_type', 'Territory':'City'})


In [ ]:
def show_info(fruits):
    print('DATASET SHAPE: ', fruits.shape, '\n')
    print('-'*50)
    print('FEATURE DATA TYPES:')
    print(fruits.info())
    print('\n', '-'*50)
    print('NUMBER OF UNIQUE VALUES PER FEATURE:', '\n')
    print(fruits.nunique())
    print('\n', '-'*50)
    print('NULL VALUES PER FEATURE')
    print(fruits.isnull().sum())
show_info(fruits)

## Cities in Italy producing fruits

In [ ]:
# Check cities names
fruits.City.unique()


In [ ]:
# Select only data for total production - quintals
fruits_total_prod = fruits[fruits['Data_type'] == 'total production - quintals ']
fruits_total_prod.head()

In [ ]:
# Transform values from quintals to tonnes
fruits_total_prod['Value'] = fruits_total_prod['Value']/10
fruits_total_prod

In [ ]:
# Rename column value to production tonnes
fruits_total_prod = fruits_total_prod.rename(columns = {'Value':'Production_tonnes'})
fruits_total_prod

In [ ]:
# drop column data type
fruits_total_prod =fruits_total_prod.drop(columns =['Data_type' ])
fruits_total_prod

In [ ]:
# Import climate dataset
climate_mean = pd.read_csv('climate_data/climate2006-2021_mean.csv')
climate_mean.head()

In [ ]:
# Check cities names
climate_mean.City.unique()

In [ ]:
# Select cities that correlate with crop data 

climate_mean = climate_mean.apply(lambda row: row[climate_mean['City'].isin(['Agrigento', 'Alessandria', 'Ancona', 'Arezzo', 'Ascoli Piceno',
                                                                             'Asti', 'Avellino', 'Bari', 'Barletta-Andria-Trani', 'Belluno',
                                                                             'Benevento', 'Bergamo', 'Biella', 'Bologna', 'Bolzano / Bozen',
                                                                             'Brescia', 'Brindisi', 'Cagliari', 'Caltanissetta', 'Campobasso',
                                                                             'Carbonia-Iglesias', 'Caserta', 'Catania', 'Catanzaro', 'Chieti',
                                                                             'Como', 'Cosenza', 'Cremona', 'Crotone', 'Cuneo', 'Enna', 'Fermo',
                                                                             'Ferrara', 'Firenze', 'Foggia', 'Forlì-Cesena', 'Frosinone',
                                                                             'Genova', 'Gorizia', 'Grosseto', 'Imperia', 'Isernia', "L'Aquila",
                                                                             'La Spezia', 'Latina', 'Lecce', 'Lecco', 'Livorno', 'Lodi',
                                                                             'Lucca', 'Macerata', 'Mantova', 'Massa-Carrara', 'Matera',
                                                                             'Medio Campidano', 'Messina', 'Milano', 'Modena',
                                                                             'Monza e della Brianza', 'Napoli', 'Novara', 'Nuoro', 'Ogliastra',
                                                                             'Olbia-Tempio', 'Oristano', 'Padova', 'Palermo', 'Parma', 'Pavia',
                                                                             'Perugia', 'Pesaro e Urbino', 'Pescara', 'Piacenza', 'Pisa',
                                                                             'Pistoia', 'Pordenone', 'Potenza', 'Prato', 'Ragusa', 'Ravenna',
                                                                             'Reggio di Calabria', "Reggio nell'Emilia", 'Rieti', 'Rimini',
                                                                             'Roma', 'Rovigo', 'Salerno', 'Sassari', 'Savona', 'Siena',
                                                                             'Siracusa', 'Sondrio', 'Sud Sardegna', 'Taranto', 'Teramo',
                                                                             'Terni', 'Torino', 'Trapani', 'Trentino Alto Adige / Südtirol',
                                                                             'Trento', 'Treviso', 'Trieste', 'Udine',
                                                                             "Valle d'Aosta / Vallée d'Aoste", 'Varese', 'Venezia',
                                                                             'Verbano-Cusio-Ossola', 'Vercelli', 'Verona', 'Vibo Valentia',
                                                                             'Vicenza', 'Viterbo'])])

climate_mean.head()

In [ ]:
climate_mean.City.unique()

In [ ]:
# Join both crop and climate datasets 
climate_fruits = pd.merge_ordered(fruits_total_prod, climate_mean,  left_by="City").fillna(0)

In [ ]:
climate_fruits.head()

Import fertilizers data 

In [ ]:
fertilizer = pd.read_csv('crops_climate_fertilizers/fertilizer2006-2021.csv',skipinitialspace=True)
fertilizer.head()

In [ ]:
crop_fertilizer_climate = crop_fertilizer_climate.drop(columns =['Unnamed: 0.1','Unnamed: 0', 'Type_fertilizer'])
crop_fertilizer_climate

Join crop and fertilizers datasets 

In [ ]:
# Join both datasets 
crop_fertilizer_climate = pd.merge(climate_fruits , fertilizer, on=['Year', 'City'], how='left').fillna(0)
crop_fertilizer_climate

In [ ]:
# Save dataset
crop_fertilizer_climate  =  crop_fertilizer_climate.to_csv('fruits_fertilizer_climate2006-2021.csv', index=False)

In [ ]:
fruits_crop  = pd.read_csv('fruits_fertilizer_climate2006-2021.csv',skipinitialspace=True)
fruits_crop .head()

Import locations lat and lon 

In [ ]:
geo = pd.read_csv('it_locations2.csv',skipinitialspace=True)
geo.head()

In [ ]:
geo.City.unique()

In [ ]:
# Select cities that correlate with crop data 

geo = geo.apply(lambda row: row[geo['City'].isin(['Agrigento', 'Alessandria', 'Ancona', 'Arezzo', 'Ascoli Piceno',
                                                                             'Asti', 'Avellino', 'Bari', 'Barletta-Andria-Trani', 'Belluno',
                                                                             'Benevento', 'Bergamo', 'Biella', 'Bologna', 'Bolzano / Bozen',
                                                                             'Brescia', 'Brindisi', 'Cagliari', 'Caltanissetta', 'Campobasso',
                                                                             'Carbonia-Iglesias', 'Caserta', 'Catania', 'Catanzaro', 'Chieti',
                                                                             'Como', 'Cosenza', 'Cremona', 'Crotone', 'Cuneo', 'Enna', 'Fermo',
                                                                             'Ferrara', 'Firenze', 'Foggia', 'Forlì-Cesena', 'Frosinone',
                                                                             'Genova', 'Gorizia', 'Grosseto', 'Imperia', 'Isernia', "L'Aquila",
                                                                             'La Spezia', 'Latina', 'Lecce', 'Lecco', 'Livorno', 'Lodi',
                                                                             'Lucca', 'Macerata', 'Mantova', 'Massa-Carrara', 'Matera',
                                                                             'Medio Campidano', 'Messina', 'Milano', 'Modena',
                                                                             'Monza e della Brianza', 'Napoli', 'Novara', 'Nuoro', 'Ogliastra',
                                                                             'Olbia-Tempio', 'Oristano', 'Padova', 'Palermo', 'Parma', 'Pavia',
                                                                             'Perugia', 'Pesaro e Urbino', 'Pescara', 'Piacenza', 'Pisa',
                                                                             'Pistoia', 'Pordenone', 'Potenza', 'Prato', 'Ragusa', 'Ravenna',
                                                                             'Reggio di Calabria', "Reggio nell'Emilia", 'Rieti', 'Rimini',
                                                                             'Roma', 'Rovigo', 'Salerno', 'Sassari', 'Savona', 'Siena',
                                                                             'Siracusa', 'Sondrio', 'Sud Sardegna', 'Taranto', 'Teramo',
                                                                             'Terni', 'Torino', 'Trapani', 'Trentino Alto Adige / Südtirol',
                                                                             'Trento', 'Treviso', 'Trieste', 'Udine',
                                                                             "Valle d'Aosta / Vallée d'Aoste", 'Varese', 'Venezia',
                                                                             'Verbano-Cusio-Ossola', 'Vercelli', 'Verona', 'Vibo Valentia',
                                                                             'Vicenza', 'Viterbo'])])
geo.head()

In [ ]:
geo.City.unique()

#### Join locations lon and lat to crop datasets 

In [ ]:
# Join both crop and locations datasets 
final_fruits_crop = pd.merge_ordered(fruits_crop, geo,  left_by="City").fillna(0)

In [ ]:
final_fruits_crop

In [1]:
# Save final dataset
final_fruits_crop  =  final_fruits_crop.to_csv('fruit_climate2006-2021.csv', index=False)

NameError: name 'final_fruits_crop' is not defined